In [62]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [31]:
url = "https://www.basketball-reference.com/awards/all_rookie.html"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

table_body = soup.find("tbody")
rows = table_body.find_all("tr", class_=False)

In [32]:
data = []
for row in rows:
    year = row.find("a").text
    if int(year[:4]) >= 2010: # filter to years since Kalipari has been head coach
        team = row.find('td', class_='left', attrs={'data-stat': 'all_team'}).text[:1]
        players = row.select('a[href^="/players"]')
        for player in players:
            # get url to player's page
            player_url = "https://www.basketball-reference.com/" + player.get('href')
            # get career advanced stats
            player_response = requests.get(player_url)
            player_soup = BeautifulSoup(player_response.content, "html.parser")
            advanced_table = player_soup.find("table", class_="stats_table", id="advanced")
            foot = advanced_table.find("tfoot")
            career_stats = []
            for td in foot.find_all("td"):
                career_stats.append(td.text)

            ranking_metrics = [career_stats[6], career_stats[22], career_stats[26], career_stats[27]]

            athlete = [year, player.text, team]
            athlete.extend(ranking_metrics)
            data.append(athlete)

            time.sleep(3)

In [65]:
draft_years = list(np.arange(2011, 2022))

[2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]

In [66]:
dfs = []

draft_years = list(np.arange(2011, 2022))

for year in draft_years:
    url = "https://www.basketball-reference.com/draft/NBA_" + str(year) + ".html"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    table_body = soup.find("tbody")
    rows = table_body.find_all("tr", class_=False)

    def parse_html(row, data_stat):
        return row.find("td", attrs={"data-stat": data_stat}).text

    data = []
    for row in rows:
        # get name, pick, college team, ws/48, bpm, vorp
        name = row.select_one('a[href^="/players/"]').text
        pick = row.find('th', class_="right", attrs={"data-stat": "ranker"}).text
        college = parse_html(row, "college_name")
        ws_per_48 = parse_html(row, "ws_per_48")
        bpm = parse_html(row, "bpm")
        vorp = parse_html(row, "vorp")

        data.append([year, name, pick, college, ws_per_48, bpm, vorp])
    
    df = pd.DataFrame(data, columns=["year", "name", "pick", "college", "ws_48", "bpm", "vorp"])
    dfs.append(df)

In [67]:
df = pd.concat(dfs)
df.to_csv("draft.csv", index=False)